In [1]:
import os
import re
import sys
from pathlib import Path

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import skimage
import squidpy as sq
from skimage import io
from sklearn import preprocessing
from tqdm.notebook import tqdm, trange
import anndata as ad
from skimage import exposure, measure, morphology, util
import cv2 

In [2]:
p_dir = (Path().cwd().parents[0].parents[0]).absolute()
data_dir = p_dir / "09_datasets"

In [3]:
dataset_1 = data_dir / '01_codex_data'
dataset_2 = data_dir / '02_tissue_architecture'
dataset_3 = data_dir / '03_Sci_Data_Cycif' 
dataset_5 = data_dir / '05_MCMICRO' / 'TNP_pilot_cycif' 

In [4]:
manual_path = data_dir / 'exported' / 'manual' 

In [5]:
# Quality control of mask
def match_gc_foll(mask_gc, mask_foll):
    '''
    Function to check if cell masks contain nuclei
    '''
    # Label foll mask
    foll = measure.label(mask_foll).astype(np.uint16)
    # Label gc mask inside foll mask
    gc = np.where(mask_gc > 0, foll, 0).astype(np.uint16)
    # Get mantel zone
    non_gc = cv2.subtract(foll, gc)
    
    return foll, gc, non_gc


def contrast_streching(img, q_min=0.5, q_max=99.5):
    p2, p98 = np.percentile(img, (q_min, q_max))
    img = exposure.rescale_intensity(img, in_range=(p2, p98), out_range=(0, 255)).astype(np.uint8)
    return img

# 01_Codex

Manually segmented all follicle and GC regions. Read both mask and assign same label to matching follicle and GC 

In [6]:
foll_mask_path = manual_path / '01_foll.png'
GC_mask_path = manual_path / '01_GC.png'

foll_mask = cv2.imread(str(foll_mask_path), cv2.IMREAD_GRAYSCALE)
foll_mask = cv2.bitwise_not(foll_mask)

GC_mask = cv2.imread(str(GC_mask_path), cv2.IMREAD_GRAYSCALE)
GC_mask = cv2.bitwise_not(GC_mask)

In [7]:
foll, gc, non_gc = match_gc_foll(GC_mask, foll_mask)

In [8]:
print(len(np.unique(foll))-1, len(np.unique(gc))-1)

88 85


In [9]:
# import napari

# napari.view_image(np.stack([foll, gc, non_gc]))

In [10]:
img_path = data_dir / 'exported' / 'foll_masks' / '01_GC.png'
io.imsave(img_path, gc)
img_path = data_dir / 'exported' / 'foll_masks' / '01_foll.png'
io.imsave(img_path, foll)
img_path = data_dir / 'exported' / 'foll_masks' / '01_nongc.png'
io.imsave(img_path, non_gc)

C:\Users\thu71\AppData\Local\Temp\ipykernel_40436\2009186643.py:2: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\01_GC.png is a low contrast image
  io.imsave(img_path, gc)
C:\Users\thu71\AppData\Local\Temp\ipykernel_40436\2009186643.py:4: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\01_foll.png is a low contrast image
  io.imsave(img_path, foll)
C:\Users\thu71\AppData\Local\Temp\ipykernel_40436\2009186643.py:6: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\01_nongc.png is a low contrast image
  io.imsave(img_path, non_gc)


# 03 Cycif

Automated segmentation of GC regions combined with manual segmentation of follicle regions

## Segment GC region by looking at combined marker expression

In [12]:
foll_mask_path = manual_path / '03_foll.png'
foll_mask = cv2.imread(str(foll_mask_path), cv2.IMREAD_GRAYSCALE)
foll_mask = cv2.bitwise_not(foll_mask)

In [13]:
img_path = data_dir / 'exported' / 'GC' / '03_GC.ome.tif'
img = io.imread(img_path, as_gray=True)
img = util.invert(img)
img = contrast_streching(img)
img = cv2.blur(img,(15,15))

In [14]:
res = cv2.bitwise_and(img,img,mask = foll_mask)

In [ ]:
img_mask = res>155
img_mask = morphology.binary_closing(img_mask, morphology.square(15))
img_mask = morphology.binary_opening(img_mask, morphology.square(7))

In [ ]:
img_mask_filtered = morphology.remove_small_holes(img_mask, 5000)
img_mask_filtered = morphology.remove_small_objects(img_mask_filtered, 2000)

In [ ]:
# import napari
# viewer = napari.view_image(res)
# viewer.add_image(img_mask)
# viewer.add_image(img_mask_filtered)

In [155]:
img_path = data_dir / 'exported' / 'GC' / '03_GC.tif'
io.imsave(img_path, img_mask_filtered)

C:\Users\thu71\AppData\Local\Temp\ipykernel_21040\1073505967.py:2: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\GC\03_GC.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(img_path, img_mask_filtered)


## Match Follicle and GC regions

In [6]:
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon 
from skimage.draw import polygon

In [7]:
foll_mask_path = manual_path / '03_foll.png'
GC_mask_path = manual_path / '03_GC.tif'

foll_mask = cv2.imread(str(foll_mask_path), cv2.IMREAD_GRAYSCALE)
foll_mask = cv2.bitwise_not(foll_mask)

GC_mask = cv2.imread(str(GC_mask_path), cv2.IMREAD_GRAYSCALE)
GC_mask = cv2.GaussianBlur(GC_mask, (0,0), sigmaX=10, sigmaY=10, borderType = cv2.BORDER_DEFAULT)


In [8]:
foll, gc, non_gc = match_gc_foll(GC_mask, foll_mask)

In [9]:
print(len(np.unique(foll))-1, len(np.unique(gc))-1)

271 190


In [10]:
# import napari

# napari.view_image(np.stack([foll, gc, non_gc]))

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 5277.0, 7494.0), zoom=0.048062529606821405, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 3.0, 1.0), (0.0, 10555.0, 1.0), (0.0, 14989.0, 1.0)), current_step=(1, 5277, 7494), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Image' at 0x213138575b0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_ver

In [11]:
img_path = data_dir / 'exported' / 'foll_masks' / '03_GC.png'
io.imsave(img_path, gc)
img_path = data_dir / 'exported' / 'foll_masks' / '03_foll.png'
io.imsave(img_path, foll)
img_path = data_dir / 'exported' / 'foll_masks' / '03_nongc.png'
io.imsave(img_path, non_gc)

C:\Users\thu71\AppData\Local\Temp\ipykernel_51076\1535520330.py:2: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\03_GC.png is a low contrast image
  io.imsave(img_path, gc)
C:\Users\thu71\AppData\Local\Temp\ipykernel_51076\1535520330.py:4: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\03_foll.png is a low contrast image
  io.imsave(img_path, foll)
C:\Users\thu71\AppData\Local\Temp\ipykernel_51076\1535520330.py:6: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\03_nongc.png is a low contrast image
  io.imsave(img_path, non_gc)


In [32]:
# cc = find_contours(blur, 200)

# cc_smooth = []
# for contour in cc:
#     cc_smooth.append(subdivide_polygon(contour, degree=2))
    
# mask1 = np.zeros(GC_mask.shape)
# for contour in cc:
#     rr, cc = polygon(contour[:, 0], contour[:, 1], mask1.shape)
#     mask1[rr, cc] = 255    

# 05 Cycif

Manually segmented all follicle and GC regions. Read both mask and assign same label to matching follicle and GC 

In [12]:
foll_mask_path = manual_path / '05_foll.png'
GC_mask_path = manual_path / '05_GC.png'

foll_mask = cv2.imread(str(foll_mask_path), cv2.IMREAD_GRAYSCALE)
foll_mask = cv2.bitwise_not(foll_mask)

GC_mask = cv2.imread(str(GC_mask_path), cv2.IMREAD_GRAYSCALE)
GC_mask = cv2.bitwise_not(GC_mask)

In [13]:
foll, gc, non_gc = match_gc_foll(GC_mask, foll_mask)

In [14]:
print(len(np.unique(foll))-1, len(np.unique(gc))-1)

92 90


In [15]:
# import napari

# napari.view_image(np.stack([foll, gc, non_gc]))

In [17]:
img_path = data_dir / 'exported' / 'foll_masks' / '05_GC.png'
io.imsave(img_path, gc)
img_path = data_dir / 'exported' / 'foll_masks' / '05_foll.png'
io.imsave(img_path, foll)
img_path = data_dir / 'exported' / 'foll_masks' / '05_nongc.png'
io.imsave(img_path, non_gc)

C:\Users\thu71\AppData\Local\Temp\ipykernel_51076\3602864885.py:2: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\05_GC.png is a low contrast image
  io.imsave(img_path, gc)
C:\Users\thu71\AppData\Local\Temp\ipykernel_51076\3602864885.py:4: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\05_foll.png is a low contrast image
  io.imsave(img_path, foll)
C:\Users\thu71\AppData\Local\Temp\ipykernel_51076\3602864885.py:6: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\05_nongc.png is a low contrast image
  io.imsave(img_path, non_gc)


# 07 NIH

In [11]:
name_pattern = ['07_T3', '07_T5', '07_T6', '07_T8' ,'07_T18', '07_T22', 
                '07_A6', '07_A8', '07_A11', '07_A18', '07_A21', '07_A22',]

for name in tqdm(name_pattern):
    foll_mask_path = manual_path / f'{name}_foll.png'
    GC_mask_path = manual_path / f'{name}_GC.png'

    foll_mask = cv2.imread(str(foll_mask_path), cv2.IMREAD_GRAYSCALE)
    foll_mask = cv2.bitwise_not(foll_mask)

    GC_mask = cv2.imread(str(GC_mask_path), cv2.IMREAD_GRAYSCALE)
    GC_mask = cv2.bitwise_not(GC_mask)

    foll, gc, non_gc = match_gc_foll(GC_mask, foll_mask)
    print(len(np.unique(foll))-1, len(np.unique(gc))-1)
    
    img_path = data_dir / 'exported' / 'foll_masks' / f'{name}_GC.png'
    io.imsave(img_path, gc)
    img_path = data_dir / 'exported' / 'foll_masks' / f'{name}_foll.png'
    io.imsave(img_path, foll)
    img_path = data_dir / 'exported' / 'foll_masks' / f'{name}_nongc.png'
    io.imsave(img_path, non_gc)

  0%|          | 0/12 [00:00<?, ?it/s]

56 52


C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:18: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_T3_GC.png is a low contrast image
  io.imsave(img_path, gc)
C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:20: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_T3_foll.png is a low contrast image
  io.imsave(img_path, foll)
C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:22: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_T3_nongc.png is a low contrast image
  io.imsave(img_path, non_gc)


45 39


C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:18: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_T5_GC.png is a low contrast image
  io.imsave(img_path, gc)
C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:20: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_T5_foll.png is a low contrast image
  io.imsave(img_path, foll)
C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:22: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_T5_nongc.png is a low contrast image
  io.imsave(img_path, non_gc)


...

C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:18: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_A18_GC.png is a low contrast image
  io.imsave(img_path, gc)
C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:20: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_A18_foll.png is a low contrast image
  io.imsave(img_path, foll)
C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:22: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_A18_nongc.png is a low contrast image
  io.imsave(img_path, non_gc)


14 10


C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:18: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_A21_GC.png is a low contrast image
  io.imsave(img_path, gc)
C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:20: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_A21_foll.png is a low contrast image
  io.imsave(img_path, foll)
C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:22: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_A21_nongc.png is a low contrast image
  io.imsave(img_path, non_gc)


28 28


C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:18: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_A22_GC.png is a low contrast image
  io.imsave(img_path, gc)
C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:20: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_A22_foll.png is a low contrast image
  io.imsave(img_path, foll)
C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\3815802243.py:22: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\07_A22_nongc.png is a low contrast image
  io.imsave(img_path, non_gc)


# 08_visinity

In [6]:
foll_mask_path = manual_path / '08_foll.png'
GC_mask_path = manual_path / '08_GC.png'

foll_mask = cv2.imread(str(foll_mask_path), cv2.IMREAD_GRAYSCALE)
foll_mask = cv2.bitwise_not(foll_mask)

GC_mask = cv2.imread(str(GC_mask_path), cv2.IMREAD_GRAYSCALE)
GC_mask = cv2.bitwise_not(GC_mask)

In [7]:
foll, gc, non_gc = match_gc_foll(GC_mask, foll_mask)

In [8]:
print(len(np.unique(foll))-1, len(np.unique(gc))-1)

40 22


In [9]:
# import napari

# napari.view_image(np.stack([foll, gc, non_gc]))

In [10]:
img_path = data_dir / 'exported' / 'foll_masks' / '08_GC.png'
io.imsave(img_path, gc)
img_path = data_dir / 'exported' / 'foll_masks' / '08_foll.png'
io.imsave(img_path, foll)
img_path = data_dir / 'exported' / 'foll_masks' / '08_nongc.png'
io.imsave(img_path, non_gc)

C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\921922722.py:2: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\08_GC.png is a low contrast image
  io.imsave(img_path, gc)
C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\921922722.py:4: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\08_foll.png is a low contrast image
  io.imsave(img_path, foll)
C:\Users\thu71\AppData\Local\Temp\ipykernel_32412\921922722.py:6: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\08_nongc.png is a low contrast image
  io.imsave(img_path, non_gc)


# Our data

In [8]:
name_pattern = ['00_LA', '00_LE', '00_TA', '00_TE']

for name in tqdm(name_pattern):
    foll_mask_path = manual_path / f'{name}_foll.png'
    GC_mask_path = manual_path / f'{name}_GC.png'

    foll_mask = cv2.imread(str(foll_mask_path), cv2.IMREAD_GRAYSCALE)
    foll_mask = cv2.bitwise_not(foll_mask)

    GC_mask = cv2.imread(str(GC_mask_path), cv2.IMREAD_GRAYSCALE)
    GC_mask = cv2.bitwise_not(GC_mask)

    foll, gc, non_gc = match_gc_foll(GC_mask, foll_mask)
    print(len(np.unique(foll))-1, len(np.unique(gc))-1)
    
    img_path = data_dir / 'exported' / 'foll_masks' / f'{name}_GC.png'
    io.imsave(img_path, gc)
    img_path = data_dir / 'exported' / 'foll_masks' / f'{name}_foll.png'
    io.imsave(img_path, foll)
    img_path = data_dir / 'exported' / 'foll_masks' / f'{name}_nongc.png'
    io.imsave(img_path, non_gc)

  0%|          | 0/4 [00:00<?, ?it/s]

14 1


C:\Users\thu71\AppData\Local\Temp\ipykernel_77948\4009400999.py:17: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\00_LA_GC.png is a low contrast image
  io.imsave(img_path, gc)
C:\Users\thu71\AppData\Local\Temp\ipykernel_77948\4009400999.py:19: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\00_LA_foll.png is a low contrast image
  io.imsave(img_path, foll)
C:\Users\thu71\AppData\Local\Temp\ipykernel_77948\4009400999.py:21: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\00_LA_nongc.png is a low contrast image
  io.imsave(img_path, non_gc)


18 8


C:\Users\thu71\AppData\Local\Temp\ipykernel_77948\4009400999.py:17: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\00_LE_GC.png is a low contrast image
  io.imsave(img_path, gc)
C:\Users\thu71\AppData\Local\Temp\ipykernel_77948\4009400999.py:19: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\00_LE_foll.png is a low contrast image
  io.imsave(img_path, foll)
C:\Users\thu71\AppData\Local\Temp\ipykernel_77948\4009400999.py:21: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\00_LE_nongc.png is a low contrast image
  io.imsave(img_path, non_gc)


23 18


C:\Users\thu71\AppData\Local\Temp\ipykernel_77948\4009400999.py:17: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\00_TA_GC.png is a low contrast image
  io.imsave(img_path, gc)
C:\Users\thu71\AppData\Local\Temp\ipykernel_77948\4009400999.py:19: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\00_TA_foll.png is a low contrast image
  io.imsave(img_path, foll)
C:\Users\thu71\AppData\Local\Temp\ipykernel_77948\4009400999.py:21: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\00_TA_nongc.png is a low contrast image
  io.imsave(img_path, non_gc)


20 16


C:\Users\thu71\AppData\Local\Temp\ipykernel_77948\4009400999.py:17: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\00_TE_GC.png is a low contrast image
  io.imsave(img_path, gc)
C:\Users\thu71\AppData\Local\Temp\ipykernel_77948\4009400999.py:19: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\00_TE_foll.png is a low contrast image
  io.imsave(img_path, foll)
C:\Users\thu71\AppData\Local\Temp\ipykernel_77948\4009400999.py:21: UserWarning: \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\09_datasets\exported\foll_masks\00_TE_nongc.png is a low contrast image
  io.imsave(img_path, non_gc)
